In [13]:
# PWM_MEMBRANE_TOY.IPYNB -  a simple toy example of PWM chemical inputs' interplane with membrane transport

# PACKAGE IMPORTS
import numpy as np
import scipy
import jax
import jax.numpy as jnp
import jaxopt
import functools
import diffrax as diffrax
from diffrax import diffeqsolve, ODETerm, SaveAt, PIDController, SteadyStateEvent

# plotting and data handling
import pandas as pd
from bokeh import plotting as bkplot, models as bkmodels, layouts as bklayouts, palettes as bkpalettes, io as bkio
from contourpy import contour_generator as cgen
import matplotlib as mpltlb

# miscellaneous
import time

# PACKAGE SETUP
# set up jax
jax.config.update('jax_platform_name', 'cpu')
jax.config.update("jax_enable_x64", True)

# set up bokeh
bkio.reset_output()
bkplot.output_notebook()

Loading BokehJS ...

In [14]:
# ODE DEFINITION
# model ODE
def ode(t, x, args):
    # unpack arguments
    par = args[0]  # model parameters

    # unpack states
    ind_int = x[0]  # intracellular inducer concentration

    # get the external inducer concentration
    ind_ext = pwm_ind_ext(t, x, args)

    # transport can be asymmetrical and depends on which concentration is greater - see Lugagne et al. 2017
    ext_greater = ind_ext > ind_int
    # rate always depends on difference between the two concentrations
    # k_in for transport INTO the cell; k_out for transport OUT of the cell
    ind_transport_rate = (par['k_in']*ext_greater + par['k_out']*(1 - ext_greater)) * (ind_ext - ind_int)

    # return the ODE (just the transport rate for the intracellular inducer variable)
    return jnp.array([ind_transport_rate])

# external inducer concentration - a PWM input
def pwm_ind_ext(t, x, args):
    # unpack arguments
    par = args[0]  # model parameters

    # get the pulse width for a set average inducer level
    pulse_width_unclipped = (par['u_avg'] - par['u_min']) / (par['u_max'] - par['u_min']) * par['duty_cycle']   # width to get desired avg ind level, regardless of feasibility
    pulse_width = jnp.clip(pulse_width_unclipped, 0, par['duty_cycle']) # clip to feasible values

     # see which inducer levels is being supplied to the system AT THE GIVEN TIME POINT
    time_into_duty_cycle = t % par['duty_cycle']    # time into the current duty cycle
    u = (time_into_duty_cycle < pulse_width) * par['u_max'] + (time_into_duty_cycle >= pulse_width) * par['u_min']  # set the inducer level based on the pulse width

    # return the inducer level
    return u


In [15]:
# MODEL PARAMETERS
par={}  # initialise

# model parameters - Lugagne et al. 2017 for IPTG
par['k_in'] = 2.75e-2  # rate constant for transport INTO the cell [1/min]
par['k_out'] = par['k_in']/2#1.11e-1    # rate constant for transport OUT of the cell [1/min]

# PWM max and min inputs
par['u_max'] = 2.0  # maximum inducer level [mM] - the maximum in Lugagne et al. 2017
par['u_min'] = 0.0  # minimum inducer level [mM] - the minimum in Lugagne et al. 2017

# PWM duty cycle duration - not specified in Lugagne et al. 2017
par['duty_cycle'] = 1.0 # duty cycle of the PWM input [min]

# user-defined average inducer level - arbitrary
par['u_avg'] = par['u_max']*0.5  # average inducer level [mM]

##
# PREPARE FOR SIMULATION

# simulation time frame
tf=(0.0,120.0)  # time frame for simulation [min]

# saving the simulation
savetimestep=par['duty_cycle']/100  # save the simulation every 0.1 min

# initial condition
x0=jnp.array([0.0])  # initial condition for the intracellular inducer concentration [mM]

# set up the ODE solver
# define the ODE term
term = ODETerm(ode)

# ODE solver and its parameters
solver = diffrax.Kvaerno3()

# define the time points at which we save the solution
stepsize_controller = PIDController(rtol=1e-6, atol=1e-6, dtmax=par['duty_cycle']/100)

In [16]:
# RUN THE SIMULATION

# run the simulation
sol = diffeqsolve(term, solver,
                  args=(par,),
                  t0=tf[0], t1=tf[1], dt0=1e-3, y0=x0,
                  saveat=SaveAt(ts=jnp.arange(tf[0], tf[1]+savetimestep, savetimestep)),
                  max_steps=None, stepsize_controller=stepsize_controller)
ts=np.array(sol.ts)
xs=np.array(sol.ys)

# get the external inducer concentrations over time (simply from PWM parameters)
us = np.array([pwm_ind_ext(t, x, (par,)) for t, x in zip(ts, xs)])

In [17]:
# FIND AVERAGE INTRACELLULAR INDUCER CONCENTRATION OVER THE LAST 10 DUTY CYCLES

# get the last 10 duty cycles
ind_int_avg_final = np.mean(xs[-10*int(par['duty_cycle']/savetimestep):,0])



In [18]:
# PLOT THE RESULTS
plot_tf=tf

# initialise figure
u_range_size=par['u_max']-par['u_min']
traj_fig = bkplot.figure(
    frame_width=720,
    frame_height=360,
    x_axis_label="Time, min",
    y_axis_label="Inducer conc., mM",
    x_range=plot_tf,
    y_range=(par['u_min']-0.1*u_range_size, par['u_max']+0.1*u_range_size),
    title='Inducer concs. over time',
    tools="box_zoom,pan,hover,reset"
)
bkplot.output_backend = "svg"
#  plot the time-averaged extracellular inducer concentration
traj_fig.hspan(par['u_avg'],
               line_width=2, line_dash='dashed', color='deeppink',
               legend_label='u_avg')
# plot the extracellular inducer concentration
traj_fig.line(ts, us,
              line_width=2, color='darkmagenta',
              legend_label='ind_ext')
# plot the intracellular inducer concentration
traj_fig.line(ts, xs[:,0],
              line_width=2, color='saddlebrown',
              legend_label='ind_int')
# add the average intracellular inducer concentration over the last 10 duty cycles
traj_fig.hspan(ind_int_avg_final,
               line_width=2, line_dash='dashed', color='darkorange',
               legend_label='ind_int_avg_final')

# add the legend
traj_fig.legend.location = "top_left"
traj_fig.legend.click_policy = "hide"

# show the plot
bkplot.show(traj_fig)